In [1]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GATConv
from torch_geometric.nn import Node2Vec
import os
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
import pickle
import csv
import os
import torch
import sqlite3
from torch_geometric.data import Data
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from torch_geometric.data import Data, DataLoader
import torch
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch_geometric.data import DataLoader
from torch_geometric.nn import GraphConv, TopKPooling, GatedGraphConv, SAGEConv, SGConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
import pickle

In [2]:
item_encoder = LabelEncoder()
user_encoder = LabelEncoder()

In [3]:
rating_encoder=LabelEncoder()

In [4]:
conn = sqlite3.connect('/home/vikash/scb-test/fsdata.db/fsdata.db')


In [5]:
checkins=pd.read_sql_query("select user_id,venue_id from checkins",conn)

In [6]:
max_checkins=checkins.groupby(['user_id']).size().reset_index(name='counts').sort_values(by='counts', ascending=False)

In [7]:
socialgraph=pd.read_sql_query("select first_user_id,second_user_id from socialgraph  where first_user_id>1 and second_user_id>1", conn)

In [8]:
ratings=pd.read_sql_query("select user_id,venue_id,rating from ratings", conn)

In [9]:
ratings

,user_id,venue_id,rating
0,1,1,5
1,1,51,4
2,1,51,2
3,1,51,5
4,1,52,5
...,...,...,...
2809575,2153498,91385,2
2809576,2153499,783,2
2809577,2153500,91385,2
2809578,2153501,68691,2


In [10]:
all_user_ids=np.union1d(ratings.user_id,checkins.user_id)
all_user_ids=np.union1d(all_user_ids,socialgraph.first_user_id)
all_user_ids=np.union1d(all_user_ids,socialgraph.second_user_id)
user_encoder.fit(all_user_ids)

LabelEncoder()

In [11]:
all_venues=np.union1d(ratings.venue_id,checkins.venue_id)
item_encoder.fit(all_venues)

LabelEncoder()

In [12]:
ratings['user_id']=user_encoder.transform(ratings.user_id)

In [13]:
ratings['venue_id']=item_encoder.transform(ratings.venue_id)

In [14]:
rating_encoder.fit(ratings.rating)
rating_values=ratings.rating.unique()

In [15]:
ratings['rating']=rating_encoder.transform(ratings.rating)

In [16]:
socialgraph['first_user_id']=user_encoder.transform(socialgraph['first_user_id'])
socialgraph['second_user_id']=user_encoder.transform(socialgraph['second_user_id'])

In [17]:
checkins['user_id']=user_encoder.transform(checkins.user_id)


In [18]:
checkins['venue_id']=item_encoder.transform(checkins.venue_id)

In [19]:
merged=pd.merge(checkins, ratings, how="left", on=["user_id", "venue_id"])

In [20]:
merged

,user_id,venue_id,rating
0,539003,1165,0
1,1328742,1165,0
2,1328743,1165,0
3,282793,1165,0
4,376543,1165,0
...,...,...,...
1597497,244603,11093,0
1597498,2150660,742,0
1597499,591823,82847,0
1597500,494679,68626,0


In [21]:
group_by_user=merged.groupby(['user_id'])

In [22]:
social_adj_lists = socialgraph.groupby('first_user_id')['second_user_id'].apply(list).to_dict()

In [ ]:
# data_list = []

# for user_id, group in tqdm(group_by_user):
#         node_features = group[['user_id','venue_id']].drop_duplicates().values
#         label = group['rating']
        
#         target_nodes=social_adj_lists.get(user_id)
#         node_features=torch.LongTensor(list(node_features)).unsqueeze(1)
#         y = torch.FloatTensor(list(label))
        
       
#         if target_nodes:
#             #users with social profile
            
#             leng=len(target_nodes)
#             source_nodes=np.full(leng, user_id)
#             edge_index = torch.tensor([source_nodes,target_nodes], dtype=torch.long)
#             edge_index = add_self_loops(edge_index, num_nodes=x.size(0))
#             data_list.append(Data(x=node_features, edge_index=edge_index, y=y))
#         else: 
#             #users without social profile
#             data_list.append(Data(x=node_features, edge_index=torch.tensor([], dtype=torch.long), y=y))


In [27]:
data_list = []

for user_id, group in tqdm(group_by_user):
        node_features = group[['user_id','venue_id']].values
        label = group['rating']
        
        target_nodes=social_adj_lists.get(user_id)
        node_features=torch.LongTensor(list(node_features)).unsqueeze(1)
        y = torch.tensor(list(label),dtype=torch.long)
        
        
        if target_nodes:
            #users with social profile
            
            leng=len(target_nodes)
            source_nodes=np.full(leng, user_id)
            edge_index = torch.tensor([source_nodes,target_nodes], dtype=torch.long)
            
            data_list.append(Data(x=node_features, edge_index=edge_index, y=y))
        else: 
            #users without social profile
            data_list.append(Data(x=node_features, edge_index=torch.tensor([], dtype=torch.long), y=y))


100%|██████████| 485381/485381 [04:21<00:00, 1855.36it/s]


In [37]:
# with open('data_list.pickle', 'wb') as handle:
#     pickle.dump(data_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('data_list.pickle', 'rb') as handle:
#     data_list = pickle.load(handle)

In [28]:
X_train, X_test = train_test_split(data_list, test_size=0.20, random_state=42)

In [29]:
X_val, X_test= train_test_split(X_test, test_size=0.50, random_state=42)

In [30]:
batch_size= 512
train_loader = DataLoader(X_train, batch_size=batch_size)
val_loader = DataLoader(X_val, batch_size=batch_size)
test_loader = DataLoader(X_test, batch_size=batch_size)

In [31]:
num_users =len(user_encoder.classes_)
num_venues = len(item_encoder.classes_)
embed_dim = 128

In [32]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = GraphConv(embed_dim * 2, 128)
        self.pool1 = TopKPooling(128, ratio=0.9)
        self.conv2 = GraphConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.9)
        self.conv3 = GraphConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.9)
        self.user_embeddings = torch.nn.Embedding(num_embeddings=num_users, embedding_dim=embed_dim)
        self.venue_embedding = torch.nn.Embedding(num_embeddings=num_venues, embedding_dim=embed_dim)        
        self.lin1 = torch.nn.Linear(256, 256)
        self.lin2 = torch.nn.Linear(256, 128)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()        
  
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        user_id = x[:,:,0]
        venue_id = x[:,:,1]
        

        emb_user = self.user_embeddings(user_id).squeeze(1)
        emb_venue = self.venue_embedding(venue_id).squeeze(1)
        
        x = torch.cat([emb_user, emb_venue], dim=1)  
#         print(x.shape,edge_index.shape)
        x = F.relu(self.conv1(x, edge_index))
        #print(x.shape)
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.act2(x)      
        
        outputs = []
        for i in range(x.size(0)):
            output = torch.matmul(emb_user[data.batch == i], x[i,:])
            outputs.append(output)
            
        x = torch.cat(outputs, dim=0)
        x = torch.sigmoid(x)
        
        return x

In [33]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['CUDA_LAUNCH_BLOCKING']= '1'
device = torch.device('cuda')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
crit = torch.nn.CrossEntropyLoss()

In [34]:
def train():
    model.train()
    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)

        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)

In [35]:
from sklearn.metrics import roc_auc_score
def evaluate(loader):
    model.eval()

    predictions = []
    labels = []

    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            pred = model(data).detach().cpu().numpy()
            label = data.y.detach().cpu().numpy()
            predictions.append(pred)
            labels.append(label)

    predictions = np.hstack(predictions)
    labels = np.hstack(labels)
    
    return roc_auc_score(labels, predictions)

In [36]:
for epoch in range(1, 200):
    loss = train()
    train_acc = evaluate(train_loader)
    val_acc = evaluate(val_loader)    
    test_acc = evaluate(test_loader)
    print('Epoch: {:03d}, Loss: {:.5f}, Train Auc: {:.5f}, Val Auc: {:.5f}, Test Auc: {:.5f}'.
          format(epoch, loss, train_acc, val_acc, test_acc))

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "/home/vikash/anaconda3/envs/pytorch3d/lib/python3.8/site-packages/torch_scatter/scatter.py", line 21, in scatter_sum
        else:
            size[dim] = int(index.max()) + 1
        out = torch.zeros(size, dtype=src.dtype, device=src.device)
              ~~~~~~~~~~~ <--- HERE
        return out.scatter_add_(dim, index, src)
    else:
RuntimeError: CUDA error: device-side assert triggered
